# Neste notebook vamos definir as funçoes de filtragem e visualizaçao dos dados a partir das classes definidas na bilbioteca GeoPandas
[Mapping and Plotting Tools](https://geopandas.org/docs/user_guide/mapping.html)

- Com as funçoes definidas e testadas neste notebook, criaremos um arquivo .py no diretorio .soureces/ para armazena-las, possibilitando assim, que possamos chama-las com a funçao import.

In [1]:
import geopandas as gpd
import fiona

import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable  # Tutorial GeoPandas Mapping and Pltting Tools

%matplotlib widget

In [2]:
# Origem do banco de dados
gdb = '/home/ggrl/geodatabase/'

## Importando um vetor do banco de dados criado no notebook passado.
- Escolha de camada e mapa
    - Caso nao tenha o mapa selecionado retorne lista de mapas disponiveis para aquela escala.
    - Caso nao escolha um mapa retorne todos os vetores daquela camada.

In [3]:
# IMPORTADOR DE LITOLOGIAS POR ESCALA --------------------------------------------------------------------------#
def importar_geometrias(camada, mapa=False):
    '''
    Recebe:
        camada      : camada vetorial base onde o objeto gdf sera plotado
        mapa        : nome do mapa presenta na camada vetorial base;

    Se retornar uma camada vazia recebera a lista das camadas veotoriais diposniveis
    Se mapa == False: retorna todos os objetos presente nesta camada vetorial
    '''

    lito =  gpd.read_file(gdb+'geodatabase.gpkg',
                        driver= 'GPKG',
                        layer= camada)
                        
    #print(f"# -- Lista de camadas vetoriais disponiveis: {list(fiona.listlayers(gdb+'geodatabase.gpkg'))}")  

    if mapa:
        folha = lito[lito.MAPA == 'Carta geológica da folha '+mapa]
        if folha.empty:
            print(f"O mapa escolhido nao est'a presente na coluna MAPA da camada veotiral.")
            print(f"# -- Lista de mapas: {list(lito.MAPA.unique())}")

        else:    
            return(folha)
    else:
        return(lito)

In [11]:
# Definindo uma funçao que importa as escaladas 
def plot_base(gdf,atributo=None,camada=None):
    '''
    Recebe:
        gdf         : objeto descrito por uma geometria e seus atributos;
        atributo    : str descritiva do atributo;
        camada      : camada vetorial base onde o objeto gdf sera plotado
    '''
    if atributo:
        ax = camada.plot('SIGLA')
        gdf.plot(column=atributo,ax=ax)
    else:
        ax = camada.plot()
        gdf.plot(ax=ax)

In [5]:
def plot_brazil(gdf,atributo=None):
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    brazil = world[world.name == 'Brazil']
    if atributo:
        ax = brazil.boundary.plot(color='black',
                                  figsize=(12,12))
        gdf.plot(column=atributo,
                 ax=ax,
                 legend=True)
    else:
        ax = brazil.boundary.plot(color='black')
        gdf.plot(ax=ax)

##  Utilizando as funçoes definidas acima para importar e visualizar os dados

In [6]:
# Abrindo os vetores descritivos dos projetos aerogeofisicos
proj_geof = importar_geometrias('proj_aerogeof')

In [12]:
litologia_100k = importar_geometrias('litologia_100k')

In [7]:
print(proj_geof.shape)

(296, 17)


In [8]:
proj_250m = proj_geof[proj_geof['ESPACAMENT'] == '250'].boundary

In [9]:
plot_base(gdf=proj_250m,camada='litologia_100k')
plt.axis('scaled')
plt.suptitle('250 metros')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, '250 metros')

In [10]:
litologia_100k

NameError: name 'litologia_100k' is not defined

In [7]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [8]:
print(len(proj_geof_500m))
proj_geof_500m['DISPONIBIL'].isnull().sum()

111


42

In [9]:
proj_geof_500m.set_geometry(col='geometry',inplace=True)

/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [10]:
proj_geof_500m.boundary.plot()
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-71.73283612029995, -33.0241609877, -34.40851261280006, 6.258765858800014)

In [11]:
# Com este plot, podemos visualizar que a area de cobertura aerogeofisica e altissima, porem,
#  precisamos analisar melhor estes dados para que possa ser utilizados nos modelos preditivos.

plot_brazil(proj_geof,atributo='ESPACAMENT')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
proj_geof.columns

Index(['ID_PROJETO', 'TITULO', 'METODOS', 'ALTURA_VOO', 'AREA_LEVAN',
       'QUILOMETRA', 'DIRECAO_LV', 'DIRECAO_LC', 'ESPACAMENT', 'ESPACAME_1',
       'ID_SERIE', 'SERIE', 'DISPONIBIL', 'IMAGENS', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry'],
      dtype='object')

In [13]:
#proj_geof.geometry.area.sum()

In [14]:
print(type(proj_geof['ESPACAMENT'][0]))
print(list(proj_geof.ESPACAMENT.unique()))

<class 'str'>
['2.5', '500', '1500', '1000', '3000', '2000', '3', '0', '250', '150', '200', '2500', '100', '4000', '2', '125', '400', '300', '1', '5', '4', '2,5', '6000', '8000', '6', '1,5', '7']


In [15]:
print(f"Foram identificados {len(proj_geof[proj_geof['ESPACAMENT'] == '500'])} projetos aerogeofisicos")
print(f"Lista de Titulos: {proj_geof[proj_geof['ESPACAMENT'] == '500']['TITULO']}")

Foram identificados 111 projetos aerogeofisicos
Lista de Titulos: 1      Projeto Aerogeofísico Escudo do Rio Grande do Sul
4           Projeto Aerogeofísico Sudeste do Mato Grosso
5              Projeto Aerogeofísico Sudeste de Rondônia
6               Projeto Aerogeofísicos Serra dos Apiacás
10     Projeto Aerogeofísico Reserva Nacional do Cobr...
                             ...                        
277                                 Serro (Área-Detalhe)
281                  Campo Alegre de Lourdes - Mortugaba
288               Projeto Aerogeofísico Pernambuco-Piauí
290                         Projeto Aerogeofísico Gurupi
295    Área 18 Minas Gerais - Porto Cajueiro - São Fr...
Name: TITULO, Length: 111, dtype: object


In [16]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [17]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geometrias('litologia_100k')
litologia_250k = importar_geometrias('litologia_250k')

In [18]:
plot_brazil(proj_geof_500m.boundary)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
#int(proj_geof['ESPACAMENT'])

In [20]:
proj_geof_500m = proj_geof[proj_geof.ESPACAMENT == '500']

In [21]:
#ax = litologia_100k.plot('SIGLA')
#proj_geof_500m.boundary.plot(ax=ax)

plot_base(proj_geof_500m.boundary,camada='litologia_100k')  # esta funçao e lenta pois importa toda a camada vetorial novamente e
                                                            # nao deve ser usada caso a litologia ja tenhasido definida

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Listando atributos geologicos das geometrias importadas.

In [22]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geologico('litologia_100k')
litologia_250k = importar_geologico('litologia_250k')

NameError: name 'importar_geologico' is not defined

In [ ]:
# Abrindo os vetores litológicos de um unico mapa utilizando a função definida acima
varginha_100k = importar_geologico('litologia_100k','Varginha - UFRJ')

In [ ]:
# atributos descritivos de cada uma das camadas litológicas (geometrias)
print("Lista de atributos descritivos classes litologicas: ")
print(list(varginha_100k.columns))

In [ ]:
# Plotando os vetores descritos pelos geólogos da cprm
varginha_100k.plot('SIGLA')

In [ ]:
print(varginha_100k.shape)
print(f"{varginha_100k.shape[0]} geometrias litologicas descritas em {varginha_100k.shape[1]} atributos")

In [ ]:
# Observando o formato da descrição da coluna litotipos
varginha_100k_litotipos = varginha_100k.LITOTIPOS.unique()
print(f"{len(list(varginha_100k_litotipos))} litotipos reconhecidos")
list(varginha_100k_litotipos)

In [ ]:
# com estas funçoes utilizadas asssimas, podemos definir uma funçao que descreve o nosso dado vetorial
def describe_geologico(gdf):
    lista_colunas = list(gdf.columns)
    lista_shape = gdf.shape
    lista_litotipos = list(gdf.LITOTIPOS.unique())
    lista_legenda = list(gdf.LEGENDA.unique())

    dic_litologico = {'lista_colunas': lista_colunas,
                      'lista_shape': lista_shape,
                      'lista_litotipos': lista_litotipos,
                      'lista_legenda': lista_legenda}
    return dic_litologico   

In [ ]:
dic_litoligico = describe_geologico(varginha_100k)

In [ ]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_litotipos'])}")
print("Lista de valores unicos da coluna Litotipo: ")
list(dic_litoligico['lista_litotipos'])

In [ ]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_legenda'])}")
print("Lista de valores unicos da coluna Legenda: ")
list(dic_litoligico['lista_legenda'])

## Reconhecendo as litologia mapeadas em suas escalas
- Faremos um filtro na coluna LEGENDA e LITOTIOPOS para selecionar as classes de interesse e, com estas geometrias, selecionaremos os mapas que as contém.
- Assim podemos identificar quais cartas geológicas e suas escalas podemos trabalhar.

In [ ]:
def filtro(gdf,mineral):
    filtrado = gdf[gdf['LITOTIPOS'].str.contains(mineral)]
    if filtrado.empty:
        print(f"{list(gdf['LITOTIPOS'].unique())}")
    else:
        return filtrado

In [ ]:
litologia_grafitosa_100k = filtro(litologia_100k,'graf')

In [ ]:
litotipos_vazio = litologia_100k[litologia_100k.LITOTIPOS.isna()]
litotipos_vazio.LITOTIPOS

In [ ]:
#Foi necessário inserir a str 'Vazio' em celas que não continham valores para que a função de filtro funcionasse corretamente na coluna LEGENDA.

#litologia_1kk.fillna('Vazio',inplace=True)
#litologia_250k.fillna('Vazio',inplace=True)
litologia_100k.fillna('Vazio',inplace=True)
#litologia_50k.fillna('Vazio',inplace=True)

In [ ]:
Grafita_l1kk = filtro(litologia_1kk,'Graf')
print(list(Grafita_l1kk['LITOTIPOS'].unique()))

In [ ]:
Grafita_l1kk.plot('SIGLA')

In [ ]:
base = brazil.boundary.plot(color='black',figsize=(12,12))

leg_grafitosa_l1kk.plot(ax=base,facecolor='grey')

lit_grafitoso_1kk.plot(ax=base,facecolor='black')

In [ ]:
list_gr_100k=mapas_lito_gr_100k
indice=0
for i in mapas_lito_gr_100k:
    print(i)
    list_gr_100k[indice] = l_100k[l_100k['MAPA'].str.contains("i")]
    indice+=1


In [ ]:
len(list_gr_100k)


### Desafio desnecessário para provar meus conhecimentos com for loops:

In [ ]:
lista_mapa = list(l_1kk.MAPA.unique())
print(len(lista_mapa))
lista_nome=[]

for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [ ]:
lista_nome

In [ ]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = l_1kk[l_1kk['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

In [ ]:
folha[31].plot()

tired for now

In [ ]:
lista_mapa = list(l_100k.MAPA.unique())
lista_mapa

In [ ]:
lista_nome=[]
for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [ ]:
len(lista_nome)

In [ ]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = l_100k[l_100k['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

In [ ]:
len(folha)